In [16]:
import pandas as pd
pd.options.mode.chained_assignment = None
import os
import requests
import re
import json


In [23]:
df = pd.read_csv("dems.csv")
df['Type'] = df['Type'].replace('Unknown', 'State')
df.head(10)

,State,Type,Name,Office,Party,Incumbent
0,Alabama,Federal,Tom Holmes,U.S. House Alabama District 1,Democratic,N
1,Alabama,Federal,Anthony Daniels,U.S. House Alabama District 2,Democratic,N
2,Alabama,Federal,Shomari Figures,U.S. House Alabama District 2,Democratic,N
3,Alabama,Federal,Elizabeth Anderson,U.S. House Alabama District 6,Democratic,N
4,Alabama,Federal,Terri Sewell,U.S. House Alabama District 7,Democratic,Y
5,Alabama,State,Tonya Smith Chestnut,Alabama State Board of Education District 5,Democratic,Y
6,Alabama,State,Greg Griffin,Alabama Supreme Court Chief Justice,Democratic,N
7,Alabama,Local,Jackie Anderson-Smith,Jefferson County Circuit Clerk,Democratic,Y
8,Alabama,Local,Angela Webb Weinberg,Jefferson County Treasurer,Democratic,N
9,Alabama,Local,Andre Bennett,Jefferson County Constable District 51,Democratic,N


In [24]:
def search_google(search_term, api_key):
    url = "https://api.scrape-it.cloud/scrape/google/serp"
    headers = {
        "x-api-key": api_key
    }
    params = {
        "q": search_term
    }
    
    response = requests.get(url, headers=headers, params=params)
    
    if response.status_code == 200:
        return response.json()
    else:
        return f"Error: {response.status_code}, {response.text}"

In [22]:
# # # Example usage:
# api_key = 'c1f8d6db-7453-4a79-a2cf-9749afe3aa80'
# linkedin_url = search_google("Buffy wicks state senate california linkedin", api_key)
# print(linkedin_url)

In [26]:
# Helper function to extract LinkedIn URLs
def get_linkedin_urls(query):
    results = search_google(query, api_key)
    if 'organicResults' in results:  # Confirm that the key exists
        # Extract the 'link' from each organic result if it's a LinkedIn URL, but only for the first 5 results
        return [res['link'] for res in results['organicResults'][:5] if 'linkedin.com/in/' in res['link']]
    else:
        # If 'organicResults' key is not present, return an empty list
        return []

def update_df_with_linkedin_urls(df, api_key):
    # Define new columns for the URLs
    df['cand_url'] = None
    df['cm_url'] = None
    df['fd_url'] = None
    
    # Iterate through each row to populate the new columns
    for index, row in df.iterrows():
        print(f"Processing row {index+1}/{len(df)}...")  # Print message before processing

        # Build search queries
        base_query = f"{row['Name']} {row['Office']} {row['State']}"
        cand_query = f"{base_query} linkedin"
        cm_query = f"{base_query} campaign manager linkedin"
        fd_query = f"{base_query} finance director linkedin"

        # Get LinkedIn URLs
        cand_urls = get_linkedin_urls(cand_query)
        cm_urls = get_linkedin_urls(cm_query)
        fd_urls = get_linkedin_urls(fd_query)

        # Assign the top LinkedIn URL from each search to the DataFrame, avoiding duplicates
        df.at[index, 'cand_url'] = cand_urls[0] if cand_urls else None
        df.at[index, 'cm_url'] = next((url for url in cm_urls if url not in cand_urls), None)
        df.at[index, 'fd_url'] = next((url for url in fd_urls if url not in cand_urls + cm_urls), None)
        
        print(f"Finished processing row {index+1}/{len(df)}.")  # Print message after processing

    return df

In [27]:
# Example usage:
df_small = df[(df['State'] == "Indiana") & (df["Type"] == "Federal")]
df_small = df_small.head(3)
api_key = 'c1f8d6db-7453-4a79-a2cf-9749afe3aa80'
df_updated = update_df_with_linkedin_urls(df_small, api_key)

Processing row 1565/3...
Finished processing row 1565/3.
Processing row 1566/3...
Finished processing row 1566/3.
Processing row 1567/3...
Finished processing row 1567/3.


In [28]:
df_updated

,State,Type,Name,Office,Party,Incumbent,cand_url,cm_url,fd_url
1564,Indiana,Federal,Marc Carmichael,U.S. Senate Indiana,Democratic,N,https://www.linkedin.com/in/spencervalentine,https://www.linkedin.com/in/marc-carmichael-13...,https://www.linkedin.com/in/mark-carmichael-08...
1565,Indiana,Federal,Valerie Lin McCray,U.S. Senate Indiana,Democratic,N,None,None,None
1566,Indiana,Federal,Frank Mrvan,U.S. House Indiana District 1,Democratic,Y,https://www.linkedin.com/in/frank-j-mrvan-a753...,https://www.linkedin.com/in/ethan-hudson-30767782,https://www.linkedin.com/in/joshua-switzer-35b...


In [20]:
# testing data types
# query = "Buffy wicks state senate california linkedin"
# results = search_google(query, api_key)

# # 'results' should be a dictionary, not a list.
# # Access the 'organicResults' list directly
# organic_results = results['organicResults']

# for organic_result in organic_results:
#     print(organic_result['link'])  # This should print the actual link

https://www.linkedin.com/in/buffy-wicks-036890bb
https://www.linkedin.com/jobs/view/field-representative-buffy-wicks-at-california-state-assembly-3851110187
https://www.linkedin.com/in/emmanuel1867
https://www.linkedin.com/posts/brilliant-corners-org_what-a-day-we-were-honored-to-join-buffy-activity-7181067057021566976-STMz
https://www.linkedin.com/posts/adawes_buffy-wicks-is-the-lead-author-for-the-california-activity-7065437352994746368-QiOe
https://en.wikipedia.org/wiki/Buffy_Wicks
https://ballotpedia.org/Buffy_Wicks
https://www.linkedin.com/posts/luisalbertomontezuma_attorney-general-bonta-assemblymember-wicks-activity-7157830687104884737-vBDy
https://a14.asmdc.org/
https://calmatters.org/california-voter-guide-2024/state-senate/
